# All detailed results

This notebooks holds all detailled results that are mentioned in the text. Re-execute to update values.

## Code

### Absolute thresholds

In [1]:
import functools

import numpy as np
import xarray as xr

TOTAL_EUROPEAN_LAND_MASS_KM2 = 4920000
THRESHOLDS = [0.005, 0.01, 0.015, 0.02, 0.03]
TECHS = ["roof", "util", "offshore"]
ALL_TECHS = TECHS + ["wind"]


def calculate_data(path_to_xy_data):
    xy = xr.open_dataset(path_to_xy_data)
    data = (
        xr
        .ones_like(xy.cost.sum("scenario"))
        .expand_dims(tech=TECHS, threshold=THRESHOLDS)
    ) * np.nan

    cost_optimal_data = xy.isel(scenario=xy.cost.argmin("scenario"))

    for tech in TECHS:
        conditions = [
            xy[other_tech] <= cost_optimal_data[other_tech]
            for other_tech in ALL_TECHS
            if other_tech != tech
        ]
        tech_mask = functools.reduce(lambda x, y: x & y, conditions)
        for threshold in THRESHOLDS:
            absolute_threshold = threshold * TOTAL_EUROPEAN_LAND_MASS_KM2
            mask = tech_mask & (xy.land_use <= absolute_threshold)
            cost = xy.where(mask).cost.min("scenario")
            data.loc[{"tech": tech, "threshold": threshold}] = (cost - cost_optimal_data.cost) / cost_optimal_data.cost
    missing_values = data.isnull()
    assert not missing_values.sel(tech="roof").any()
    assert not missing_values.sel(tech="util").any()
    assert missing_values.sel(tech="offshore", threshold=0.01).sum() < 20 # in rare cases, 1% can not be done
    assert missing_values.sel(tech="offshore", threshold=0.005).sum() < 1000 # in 1% of cases, 0.05% can not be done
    return data

### Cost-effective phase-ins

In [2]:
from dataclasses import dataclass
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt

EPSILON = 0.00001
GREY = "#7F7F7F"
BLUE = "#4F6DB8"
BRIGHT_COLOR = sns.light_palette(BLUE, 3)[0]
DARK_COLOR = BLUE
PANEL_FONT_SIZE = 10
PANEL_FONT_WEIGHT = "bold"

idx = pd.IndexSlice


@dataclass
class PlotData:
    name: str
    data: pd.DataFrame
    optimal_path: pd.Series
    average_slope: float
    ylabel: str = "Cost relative to\n cost minimal case"
    xlabel: str = "Land requirements relative\nto cost minimal case"


def read_phase_ins(path_to_data):
    data = (
        xr
        .open_dataset(path_to_data)
        .mean("sample_id")
        .to_dataframe()
        .set_index(["util", "wind", "roof", "offshore"])
    )
    rel_data = data / data.loc[data.cost.idxmin()]

    return [
        PlotData(
            name="a - Offshore wind",
            data=rel_data,
            optimal_path=optimal_path_series(data, "offshore"),
            average_slope=average_slope(data, optimal_path_series(data, "offshore"))
        ),
        PlotData(
            name="b - Utility-scale PV",
            data=rel_data,
            optimal_path=optimal_path_series(data, "util"),
            average_slope=average_slope(data, optimal_path_series(data, "util"))
        ),
        PlotData(
            name="c - Rooftop PV",
            data=rel_data,
            optimal_path=optimal_path_series(data, "roof"),
            average_slope=average_slope(data, optimal_path_series(data, "roof"))
        )
    ]


def slope(index1, index2, data):
    cost_delta = data.loc[index1, "cost"] - data.loc[index2, "cost"]
    land_use_delta = (data.loc[index1, "land_use"] - data.loc[index2, "land_use"]) * 1e6 # to m2
    if abs(land_use_delta) > EPSILON:
        return cost_delta / land_use_delta
    else:
        return None


def optimal_path(data, tech):
    assert tech in ["util", "wind", "roof", "offshore"]
    current_index = data["cost"].idxmin() # start at cost minimum
    yield current_index, 0
    while data.loc[[current_index]].index.get_level_values(tech)[0] < 100:
        if tech == "util":
            index = idx[current_index[0] + 10, :current_index[1], :current_index[2], :current_index[3]]
        elif tech == "wind":
            index = idx[:current_index[0], current_index[1] + 10, :current_index[2], :current_index[3]]
        elif tech == "roof":
            index = idx[:current_index[0], :current_index[1], :current_index[2] + 10, :current_index[3]]
        elif tech == "offshore":
            index = idx[:current_index[0], :current_index[1], :current_index[2], current_index[3] + 10]
        slopes = (
            data
            .sort_index(level=['util', 'wind', 'roof', 'offshore'])
            .loc[index, ]
            .apply(lambda row: slope(current_index, row.name, data), axis=1)
        )
        slopes = slopes[slopes < 0] # don't follow slopes that lead to worse points
        if len(slopes.index) == 0:
            return
        else:
            optimal_index = slopes.abs().idxmin()
            yield optimal_index, slopes.loc[optimal_index]
            current_index = optimal_index


def optimal_path_series(data, tech):
    slopes = pd.Series({index: slope for index, slope in optimal_path(data, tech)})
    slopes.index = slopes.index.set_names(["util", "wind", "roof", "offshore"])
    return slopes


def average_slope(data, optimal_path):
    cost_optimal_index = data["cost"].idxmin()
    if len(optimal_path.index) > 0:
        return slope(cost_optimal_index, optimal_path.index[-1], data)
    else:
        return None

## Expected results

In [3]:
xy = xr.open_dataset("../build/euler/output/national/land-use/xy.nc")

In [4]:
expected = xy.mean("sample_id")

In [5]:
cost_optimal_expected_scenario = expected.to_dataframe().cost.idxmin()

In [6]:
total = expected.sel(scenario=cost_optimal_expected_scenario).land_use.item()
relative = total / TOTAL_EUROPEAN_LAND_MASS_KM2 * 100
print(f"Expected land use of cost-optimal system design using expected cost:")
print(f"{total:.0f} km2 ({relative:.1f}%)")

Expected land use of cost-optimal system design using expected cost:
96700 km2 (2.0%)


## Wind PV systems

In [7]:
df = xy.where((xy.roof == 0) & (xy.offshore == 0) & (xy.wind == 50), drop=True).to_dataframe()
df.land_use = df.land_use / TOTAL_EUROPEAN_LAND_MASS_KM2 * 100

In [8]:
df.land_use.describe()

count    99999.000000
mean         1.965441
std          0.506600
min          1.069894
25%          1.639552
50%          1.862239
75%          2.164230
max         12.466542
Name: land_use, dtype: float64

## Land limits

In [9]:
penalties = calculate_data("../build/euler/output/national/land-use/xy.nc")

In [10]:
for threshold in THRESHOLDS:
    filtered_penalities = penalties.sel(threshold=threshold, tech="util")
    share_penalty = ((filtered_penalities > 0).sum() / filtered_penalities.count()).item() * 100
    print(f"Limit of {threshold * 100:.1f}% using any tech requires penalities in {share_penalty:.0f}% of the time.")

Limit of 0.5% using any tech requires penalities in 79% of the time.
Limit of 1.0% using any tech requires penalities in 76% of the time.
Limit of 1.5% using any tech requires penalities in 63% of the time.
Limit of 2.0% using any tech requires penalities in 33% of the time.
Limit of 3.0% using any tech requires penalities in 5% of the time.


In [11]:
for tech in TECHS:
    for threshold in THRESHOLDS:
        filtered_penalities = penalties.sel(threshold=threshold, tech=tech)
        penalty = filtered_penalities.mean().item() * 100
        print(f"Limit of {threshold * 100:.1f}% using {tech} requires an expected penality of {penalty:.0f}%.")
    print()

Limit of 0.5% using roof requires an expected penality of 28%.
Limit of 1.0% using roof requires an expected penality of 17%.
Limit of 1.5% using roof requires an expected penality of 9%.
Limit of 2.0% using roof requires an expected penality of 3%.
Limit of 3.0% using roof requires an expected penality of 0%.

Limit of 0.5% using util requires an expected penality of 21%.
Limit of 1.0% using util requires an expected penality of 12%.
Limit of 1.5% using util requires an expected penality of 5%.
Limit of 2.0% using util requires an expected penality of 1%.
Limit of 3.0% using util requires an expected penality of 0%.

Limit of 0.5% using offshore requires an expected penality of 13%.
Limit of 1.0% using offshore requires an expected penality of 8%.
Limit of 1.5% using offshore requires an expected penality of 5%.
Limit of 2.0% using offshore requires an expected penality of 2%.
Limit of 3.0% using offshore requires an expected penality of 0%.



## Cost-effective Phase-ins

In [12]:
data = read_phase_ins("../build/euler/output/national/land-use/xy.nc")

In [13]:
import scipy

def interpolate_cost(data, share1, share2, new_land_use):
    new_cost = scipy.interpolate.interp1d(
        [data.loc[share1, "land_use"], data.loc[share2, "land_use"]],
        [data.loc[share1, "cost"], data.loc[share2, "cost"]]
    )(new_land_use).item()
    print(f"Interpolated cost at land_use={new_land_use}: {new_cost:.2f}")

### Offshore

In [14]:
data[0].optimal_path

util  wind  roof  offshore
50    50    0     0            0.000000
      40    0     10          -0.198586
      30    0     20          -0.231433
      20    0     30          -0.264294
      10    0     40          -0.292020
      0     0     50          -0.314384
40    0     0     60          -5.992023
30    0     0     70         -28.504441
dtype: float64

In [15]:
interpolate_cost(data[0].data, (50, 20, 0, 30), (50, 30, 0, 20), new_land_use=0.5)

Interpolated cost at land_use=0.5: 1.05


In [16]:
data[0].data.loc[(50, 0, 0, 50)]

land_use    0.149673
cost        1.096182
Name: (50, 0, 0, 50), dtype: float64

### Utility-scale PV

In [17]:
data[1].optimal_path

util  wind  roof  offshore
50    50    0     0           0.000000
60    40    0     0          -0.153239
70    30    0     0          -0.402116
80    20    0     0          -0.559840
90    10    0     0          -0.605407
100   0     0     0          -1.113324
dtype: float64

In [18]:
interpolate_cost(data[0].data, (90, 10, 0, 0), (80, 20, 0, 0), new_land_use=0.5)

Interpolated cost at land_use=0.5: 1.10


In [19]:
data[1].data.loc[(100, 0, 0, 0)]

land_use    0.193638
cost        1.225843
Name: (100, 0, 0, 0), dtype: float64

## Rooftop-PV

In [20]:
data[2].optimal_path

util  wind  roof  offshore
50    50    0     0           0.000000
      40    10    0          -0.682708
      30    20    0          -0.855521
      20    30    0          -0.759081
      10    40    0          -1.071837
      0     50    0          -1.775962
40    0     60    0          -4.756840
30    0     70    0          -5.010399
20    0     80    0          -5.051814
10    0     90    0          -5.155228
0     0     100   0          -5.649277
dtype: float64

In [21]:
interpolate_cost(data[0].data, (50, 20, 30, 0), (50, 30, 20, 0), new_land_use=0.5)

Interpolated cost at land_use=0.5: 1.17


In [22]:
data[1].data.loc[(0, 0, 100, 0)]

land_use    0.000000
cost        1.623707
Name: (0, 0, 100, 0), dtype: float64

## Balancing

When considering 10^5^\ Wh/m^2^ for battery storage capacity, a conservative 6,000\ Wh/m^2^ for hydrogen storage capacity, and 10^4^\ W/m^2^ for bioenergy capacity (see Methods), total land requirements of all three flexibility technologies are always below 1,800\ km^2^ (0.04% of total European land).

In [23]:
data = xr.open_dataset("../build/euler/output/national/aggregation.nc").sum("locs")

In [24]:
max_battery = data.sel(techs="battery").storage_cap.max().item()
max_hydrogen = data.sel(techs="hydrogen").storage_cap.max().item()
max_bio = data.sel(techs="biofuel").energy_cap.max().item()

In [25]:
max_land_battery = max_battery / 1e5
max_land_hydrogen = max_hydrogen / 6000
max_land_bioenergy = max_bio / 1e4

In [26]:
print(f"Battery: {max_land_battery:.0f} km2")
print(f"Hydrogen: {max_land_hydrogen:.0f} km2")
print(f"Bioenergy: {max_land_bioenergy:.0f} km2")
print(f"Total: {max_land_battery + max_land_hydrogen + max_land_bioenergy:.0f} km2")

Battery: 18 km2
Hydrogen: 1719 km2
Bioenergy: 24 km2
Total: 1760 km2
